### Rideshare Simulation

## Overview
- Riders and drivers are placed randomly across all available zones with equal probability.
- The matching algorithm is designed to find the nearest available driver for a rider's ride request. The driver must be in the 'idle' state and willing to accept the ride based on their preferences and the ride's metadata.

## Matching 

### 1. **Sort drivers**
First, the algorithm sorts all drivers based on their distance from the rider's current zone. This sorting is done using the Manhattan distance between the driver's zone and the rider's zone. This ensures that the closest drivers are considered first.

### 2. **Iterate through drivers**
Next, the algorithm iterates through the sorted list of drivers. For each driver, it performs the following checks:

#### a. **Check driver's state**
The driver must be in the 'idle' state, meaning they are available to take a new ride.

#### b. **Calculate & compare acceptance threshold**
The driver calculates an acceptance threshold for the ride based on their preferences and the ride's metadata. This threshold is determined by a weighted sum of the ride's attributes (cost, distance, duration, driver rating, and rider rating). The calculated acceptance threshold is compared with the driver's predefined acceptance threshold. If the calculated threshold is greater than or equal to the driver's acceptance threshold, the driver is willing to accept the ride.

### 3. **Assign driver**
If an idle driver willing to accept the ride is found, the algorithm returns this driver along with the calculated travel time to the rider's zone. If no such driver is found, the ride request may need to be re-evaluated with modified parameters (e.g., increased cost) until a suitable driver is found.

---  

The rideshare simulation consists of three main classes: `Zone`, `Driver`, `Rider`, and `RideRequest`. These classes work together to simulate the rideshare system.

In [1]:
import random
from datetime import datetime, timedelta
from transitions import Machine


The `Zone` class represents a geographical area within the grid system. Each zone has a unique identifier based on its x and y coordinates.

In [2]:
# Define the Zone class
class Zone:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.name = f"Zone ({x},{y})"


The `Driver` class represents a driver in the rideshare system. Each driver is associated with a specific `Zone` and has preferences for accepting ride requests. The driver's state is managed by a state machine, which transitions between different states such as 'idle', 'en_route', 'waiting_for_rider', 'in_ride', and 'ride_completed'.

In [3]:
# Define the Driver class with state machine
class Driver:
    states = ['idle', 'en_route', 'waiting_for_rider', 'in_ride', 'ride_completed']

    def __init__(self, name, zone, preferences):
        self.name = name
        self.zone = zone
        self.preferences = preferences  # Driver's preferences for ride attributes
        self.time_to_next_zone = 0
        self.current_rider = None
        self.current_request = None
        self.machine = Machine(model=self, states=Driver.states, initial='idle')
        self.machine.add_transition('accept_ride', 'idle', 'en_route')
        self.machine.add_transition('arrive_at_pickup', 'en_route', 'waiting_for_rider')
        self.machine.add_transition('pick_up_rider', 'waiting_for_rider', 'in_ride')
        self.machine.add_transition('complete_ride', 'in_ride', 'ride_completed')
        self.machine.add_transition('reset', '*', 'idle')

    # Calculate acceptance threshold based on ride metadata and driver preferences
    def calculate_acceptance_threshold(self, ride_metadata):
        threshold = (
            ride_metadata['cost_usd'] * self.preferences['cost_weight'] +
            ride_metadata['distance_km'] * self.preferences['distance_weight'] +
            ride_metadata['duration_min'] * self.preferences['duration_weight'] +
            ride_metadata['driver_rating'] * self.preferences['driver_rating_weight'] +
            ride_metadata['rider_rating'] * self.preferences['rider_rating_weight']
        )
        return threshold

    # Determine if the driver accepts the ride based on the calculated threshold
    def will_accept_ride(self, ride_metadata):
        acceptance_threshold = self.calculate_acceptance_threshold(ride_metadata)
        print("Acceptance threshold:", acceptance_threshold)
        print("Self threshold:", self.preferences['acceptance_threshold'])
        return acceptance_threshold >= self.preferences['acceptance_threshold']

    def __str__(self):
        return f"{self.name} (State: {self.state}, Zone: {self.zone.name}, Time to next zone: {self.time_to_next_zone})"



The `Rider` class represents a rider in the rideshare system. Each rider is also associated with a specific `Zone` and has an estimated wait time for being matched with a driver. The rider's state is managed by a state machine, which transitions between states like 'waiting', 'matched', 'in_ride', 'ride_completed', and 'ride_canceled'.


In [4]:
# Define the Rider class with state machine
class Rider:
    states = ['waiting', 'matched', 'in_ride', 'ride_completed', 'ride_canceled']

    def __init__(self, name, zone):
        self.name = name
        self.zone = zone
        self.estimated_wait_time = 0
        self.machine = Machine(model=self, states=Rider.states, initial='waiting')
        self.machine.add_transition('match_with_driver', 'waiting', 'matched')
        self.machine.add_transition('start_ride', 'matched', 'in_ride')
        self.machine.add_transition('complete_ride', 'in_ride', 'ride_completed')
        self.machine.add_transition('cancel_ride', '*', 'ride_canceled')

    def __str__(self):
        return f"{self.name} (State: {self.state}, Zone: {self.zone.name}, Estimated wait time: {self.estimated_wait_time})"



The `RideRequest` class represents a ride request made by a rider. It connects a `Rider` with a `Driver` and includes information about the origin `Zone`, destination `Zone`, and the `Zone` of the driver at the time of the request. The ride request also contains metadata such as distance, duration, cost, ratings, ride type, traffic conditions, and weather conditions. The state of the ride request is managed by a state machine, transitioning between states like 'requested', 'accepted', 'in_progress', 'completed', and 'canceled'.


In [5]:
# Define the RideRequest class with state machine
class RideRequest:
    states = ['requested', 'accepted', 'in_progress', 'completed', 'canceled']

    def __init__(self, rider, origin_zone, destination_zone, driver_zone):
        self.rider = rider
        self.origin_zone = origin_zone
        self.destination_zone = destination_zone
        self.driver_zone = driver_zone
        self.metadata = self.generate_metadata()
        self.machine = Machine(model=self, states=RideRequest.states, initial='requested')
        self.machine.add_transition('accept', 'requested', 'accepted')
        self.machine.add_transition('start', 'accepted', 'in_progress')
        self.machine.add_transition('complete', 'in_progress', 'completed')
        self.machine.add_transition('cancel', '*', 'canceled')

    # Generate metadata for the ride request
    def generate_metadata(self):
        driver_to_pickup = calculate_travel_time((self.driver_zone.x, self.driver_zone.y), (self.origin_zone.x, self.origin_zone.y))
        pickup_to_destination = calculate_travel_time((self.origin_zone.x, self.origin_zone.y), (self.destination_zone.x, self.destination_zone.y))
        total_distance = driver_to_pickup + pickup_to_destination
        duration = total_distance  # 1 unit per zone
        cost = total_distance * 2.5  # Cost is based on the total distance with a multiplier
        return {
            "distance_km": total_distance,
            "duration_min": duration,
            "pickup_time": None,
            "dropoff_time": None,
            "cost_usd": round(cost, 2),
            "driver_rating": round(random.uniform(4.0, 5.0), 2),
            "rider_rating": round(random.uniform(4.0, 5.0), 2),
            "ride_type": random.choice(["economy", "premium", "pool"]),
            "traffic_conditions": random.choice(["normal", "heavy", "light"]),
            "weather_conditions": random.choice(["clear", "rainy", "stormy", "snowy"]),
        }

    def __str__(self):
        return f"RideRequest for {self.rider.name} (\nOrigin: {self.origin_zone.name}, \nDestination: {self.destination_zone.name}, \nState: {self.state}, \nMetadata: {self.metadata})"



The simulation workflow involves the interaction between these classes. When a `Rider` makes a `RideRequest`, the system searches for an available `Driver` in the nearby `Zones`. The `Driver`'s acceptance of the ride request is determined by their preferences and the ride request metadata. If a `Driver` accepts the request, the states of the `Rider`, `Driver`, and `RideRequest` transition accordingly, simulating the progress of the ride from the origin `Zone` to the destination `Zone`.

The `Zone` class serves as a geographical context for the `Driver`s, `Rider`s, and `RideRequest`s, enabling the calculation of distances, travel times, and other location-based metrics.

Overall, the `Driver`, `Rider`, and `RideRequest` classes work together to simulate the rideshare system, with the `Zone` class providing the geographical context for their interactions.


In [6]:
# Function to calculate Manhattan distance
def calculate_travel_time(zone1, zone2):
    return abs(zone1[0] - zone2[0]) + abs(zone1[1] - zone2[1])

# Function to find the nearest available driver who accepts the ride based on their preferences
def find_nearest_driver(rider_zone, ride_metadata):
    sorted_drivers = sorted(drivers, key=lambda d: calculate_travel_time((d.zone.x, d.zone.y), (rider_zone.x, rider_zone.y)))
    for driver in sorted_drivers:
        will_accept = driver.will_accept_ride(ride_metadata)
        print("Will accept:", will_accept)
        if driver.state == 'idle' and will_accept:
            return driver, calculate_travel_time((driver.zone.x, driver.zone.y), (rider_zone.x, rider_zone.y))
    return None, None

Here's a simulation workflow:

In [7]:
# Here, we create a grid of zones, with each zone represented by its coordinates (x, y).
# This grid spans from (1,1) to (10,10), creating a total of 100 zones.
zones = {(x, y): Zone(x, y) for x in range(1, 11) for y in range(1, 11)}

# We define the specific zones where the rider, driver, and the ride destination are located.
rider_zone = zones[(2, 7)]
driver_zone = zones[(5, 5)]
destination_zone = zones[(5, 7)]
driver_preferences = {
    "cost_weight": 0.4, 
    "distance_weight": 0.1, 
    "duration_weight": 0.1, 
    "driver_rating_weight": 0.2, 
    "rider_rating_weight": 0.2, 
    "acceptance_threshold": 20
},

# Initialize 1 rider and 1 driver with specific zones and preferences
# The driver_preferences dictionary contains weights for various ride attributes such as cost, distance, duration, driver rating, and rider rating.
# The acceptance_threshold is a threshold value that the driver uses to decide whether to accept a ride request.
riders = [Rider('Rider 0', rider_zone)]
drivers = [Driver('Driver 0', driver_zone, driver_preferences[0])]
ride_request = RideRequest(riders[0], rider_zone, destination_zone, driver_zone)
ride_requests = [ride_request]

In [8]:
# Simulation workflow
def simulate_rideshare(price_increment=5):
    time_unit = 0

    while any(rider.state != 'ride_completed' for rider in riders):
        print(f"\nTime Unit: {time_unit}")

        for rider in riders:
            if rider.state == 'waiting':
                ride_request = next((request for request in ride_requests if request.rider == rider), None)
                if ride_request:
                    current_cost = ride_request.metadata["cost_usd"]
                    driver, travel_time = find_nearest_driver(rider.zone, ride_request.metadata)
                    while not driver:
                        current_cost += price_increment
                        ride_request.metadata["cost_usd"] = current_cost
                        driver, travel_time = find_nearest_driver(rider.zone, ride_request.metadata)

                    if driver:
                        print(f"{driver.name} will pick up {rider.name} from {rider.zone.name}. Travel time: {travel_time}")
                        driver.zone = rider.zone
                        driver.time_to_next_zone = travel_time
                        driver.current_rider = rider
                        driver.current_request = ride_request
                        driver.accept_ride()  # This should trigger the transition to 'en_route'
                        driver.current_request.metadata["pickup_time"] = datetime.now() + timedelta(minutes=time_unit)
                        driver.current_request.accept()
                        rider.match_with_driver()
                        rider.estimated_wait_time = travel_time

        for driver in drivers:
            print("Driver State:", driver)
            if driver.time_to_next_zone > 0:
                driver.time_to_next_zone -= 1
                print(f"{driver.name} is traveling. Time to next zone: {driver.time_to_next_zone}")
            elif driver.time_to_next_zone == 0:
                if driver.is_en_route():
                    driver.arrive_at_pickup()
                    driver.zone = driver.current_rider.zone
                    print(f"{driver.name} has arrived at the pickup zone for {driver.current_rider.name}.")
                elif driver.is_waiting_for_rider():
                    driver.pick_up_rider()
                    driver.current_request.start()
                    driver.current_rider.start_ride()
                    driver.time_to_next_zone = calculate_travel_time((driver.zone.x, driver.zone.y), (driver.current_request.destination_zone.x, driver.current_request.destination_zone.y))
                    print(f"{driver.name} picked up {driver.current_rider.name} and is now in ride.")
                elif driver.is_in_ride():
                    if driver.time_to_next_zone == 0:
                        driver.complete_ride()
                        driver.current_request.metadata["dropoff_time"] = datetime.now() + timedelta(minutes=time_unit)
                        driver.current_request.complete()
                        driver.current_rider.complete_ride()
                        print(f"{driver.name} completed the ride with {driver.current_rider.name}.")
                        driver.current_rider = None
                        driver.current_request = None
                        driver.reset()
                        print(f"{driver.name} is now idle.")
        time_unit += 1

In [9]:
# Show the initial state of rides / drivers
for rider in riders:
    print(rider)
for driver in drivers:
    print(driver)
for request in ride_requests:
    print(request)

Rider 0 (State: waiting, Zone: Zone (2,7), Estimated wait time: 0)
Driver 0 (State: idle, Zone: Zone (5,5), Time to next zone: 0)
RideRequest for Rider 0 (
Origin: Zone (2,7), 
Destination: Zone (5,7), 
State: requested, 
Metadata: {'distance_km': 8, 'duration_min': 8, 'pickup_time': None, 'dropoff_time': None, 'cost_usd': 20.0, 'driver_rating': 4.25, 'rider_rating': 4.66, 'ride_type': 'pool', 'traffic_conditions': 'heavy', 'weather_conditions': 'snowy'})


In [10]:
simulate_rideshare()


Time Unit: 0
Acceptance threshold: 11.382000000000001
Self threshold: 20
Will accept: False
Acceptance threshold: 13.382000000000001
Self threshold: 20
Will accept: False
Acceptance threshold: 15.382000000000001
Self threshold: 20
Will accept: False
Acceptance threshold: 17.382
Self threshold: 20
Will accept: False
Acceptance threshold: 19.382
Self threshold: 20
Will accept: False
Acceptance threshold: 21.382
Self threshold: 20
Will accept: True
Driver 0 will pick up Rider 0 from Zone (2,7). Travel time: 5
Driver State: Driver 0 (State: en_route, Zone: Zone (2,7), Time to next zone: 5)
Driver 0 is traveling. Time to next zone: 4

Time Unit: 1
Driver State: Driver 0 (State: en_route, Zone: Zone (2,7), Time to next zone: 4)
Driver 0 is traveling. Time to next zone: 3

Time Unit: 2
Driver State: Driver 0 (State: en_route, Zone: Zone (2,7), Time to next zone: 3)
Driver 0 is traveling. Time to next zone: 2

Time Unit: 3
Driver State: Driver 0 (State: en_route, Zone: Zone (2,7), Time to nex